# Multi i-TED characterization
## Fifth: Hardware updates

- In the [previous notebook](./PyROOT_resolution_Cs137v4.html) hardware updates were proposed in order to:
    - Fix light distributions effects
    - Improve the resolution of the worst crystals
- The current setup is considered the "final" multi i-TED system
- Characteristics of the analysis:
    - Normalized
    - Background subtracted was not applied beacause the measurements were of 30s
    - Calibrated with 88c, 100ns
    - Thresholds: 888 and 88c
    - Integration windows: 100, 150, 200, 250 ns
    - 25 runs per measurements per i-TED
- Analysis:
    - Resolution
    - Counting rate
    - Alpha activity

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

import glob

uproot              5.0.7
Welcome to JupyROOT 6.28/02
sklearn             1.2.2
torch               2.0.0
scipy               1.10.1
numpy               1.23.5
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.62.3


In [2]:
%jsroot

In [3]:
class spectrum:
    
    def __init__(self, File_, iTED_, Crystal_, Configuration_, Window_, Calibration_, Run_):  

        self.__File = ROOT.TFile.Open(File_,"READ")
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Window = Window_
        self.__Calibration = Calibration_
        self.__Run = Run_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def __ch__(self, en):
        p = numpy.poly1d(self.__Calibration[::-1])
        temp  = set(i for i in (p - en).roots if i > 0)
        return list(temp).pop()

    def File(self):
        return self.__File
    
    def TH1D(self):
        self.__TH1D = self.__File.Get(
                            "{}_{}_amplitude_spectra;1".format(
                                "SCATTERER" if self.__Crystal==0 else "ABSORBER",
                                self.__iTED if self.__Crystal==0 else "{}_{}".format(self.__iTED,self.__Crystal)
                            )
                        )
        
        return self.__TH1D
    
    def Calibration(self):
        return self.__Calibration
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Window(self):
        return self.__Window
    
    def Rate(self, Time_):
        return self.__TH1D.Integral()/Time_
    
    def Alpha(self, Time_):
        return self.__TH1D.Integral(
            self.__TH1D.FindBin(self.__ch__(1600)),
            self.__TH1D.FindBin(self.__ch__(2800))
        )/Time_
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {},Window: {}, Run: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window,
            self.__Run
        )
    
    def __str__(self):
        return "{}.{}.{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window,
            self.__Run
        )

In [4]:
calp = glob.glob('/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/Calibration/**/*.CALp', recursive=True)

calibrations = []

for i in calp:
    line = pandas.read_csv(
        i,
        sep = "      :  ",
        skiprows=[0,4,5,6],
        header=None,
        engine="python"
    ).drop([0], axis=1).T
    
    line["crystal"] = i.split("/")[-1][:2]
    line["cw"] = i.split("/")[-1][5:8]
    
    calibrations.append(line)
    
calibrations_df = pandas.concat(calibrations)

In [5]:
config = ["888","88c"]
CW = [100,150,200,250]
run = [1,2,3,4,5]

ited = ["A","B","C","D"]
crystal = [0,1,2,3,4]

run = range(2,5+1)

midx = pandas.MultiIndex.from_product(
    [run, config, CW],
    names=['Run', 'Configuration', 'Window']
)

mcol = pandas.MultiIndex.from_product(
    [ited, crystal],
    names=['iTED', 'Crystal']
) 

spectra = pandas.DataFrame(index = midx, columns = mcol)

# Access using spectra[iTED,Crystal][Configuration,Window]

In [6]:
iTEDA = {
    "888_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_31_00_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie1_25_CW{win}.root",
    "888_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_31_34_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie2_25_CW{win}.root",
    "888_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_32_07_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie3_25_CW{win}.root",
    "888_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_32_41_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie4_25_CW{win}.root",
    "888_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_33_15_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie5_25_CW{win}.root",
    "888_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_33_48_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie6_25_CW{win}.root",
    "888_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_34_22_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie7_25_CW{win}.root",
    "888_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_34_56_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie8_25_CW{win}.root",
    "888_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_35_29_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie9_25_CW{win}.root",
    "888_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_36_03_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie10_25_CW{win}.root",
    "888_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_36_37_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie11_25_CW{win}.root",
    "888_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_37_11_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie12_25_CW{win}.root",
    "888_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_37_44_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie13_25_CW{win}.root",
    "888_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_38_18_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie14_25_CW{win}.root",
    "888_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_38_52_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie15_25_CW{win}.root",
    "888_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_39_26_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie16_25_CW{win}.root",
    "888_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_39_59_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie17_25_CW{win}.root",
    "888_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_40_33_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie18_25_CW{win}.root",
    "888_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_41_07_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie19_25_CW{win}.root",
    "888_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_41_40_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie20_25_CW{win}.root",
    "888_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_42_14_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie21_25_CW{win}.root",
    "888_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_42_48_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie22_25_CW{win}.root",
    "888_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_43_21_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie23_25_CW{win}.root",
    "888_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_43_55_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie24_25_CW{win}.root",
    "888_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDA_HWPlus_D.2023_05_03_T.16_44_29_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie25_25_CW{win}.root",
    "88c_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDA_HWPlus_D.2023_05_08_T.15_54_47_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_5_CW{win}.root",
    "88c_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDA_HWPlus_D.2023_05_08_T.15_55_21_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_5_CW{win}.root",
    "88c_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDA_HWPlus_D.2023_05_08_T.15_55_55_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_5_CW{win}.root",
    "88c_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDA_HWPlus_D.2023_05_08_T.15_56_29_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_5_CW{win}.root",
    "88c_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDA_HWPlus_D.2023_05_08_T.15_57_02_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_5_CW{win}.root",
    "88c_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_01_46_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_20_CW{win}.root",
    "88c_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_02_20_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_20_CW{win}.root",
    "88c_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_02_54_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_20_CW{win}.root",
    "88c_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_03_27_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_20_CW{win}.root",
    "88c_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_04_01_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_20_CW{win}.root",
    "88c_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_04_35_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie6_20_CW{win}.root",
    "88c_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_05_09_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie7_20_CW{win}.root",
    "88c_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_05_42_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie8_20_CW{win}.root",
    "88c_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_06_16_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie9_20_CW{win}.root",
    "88c_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_06_50_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie10_20_CW{win}.root",
    "88c_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_07_24_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie11_20_CW{win}.root",
    "88c_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_07_57_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie12_20_CW{win}.root",
    "88c_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_08_31_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie13_20_CW{win}.root",
    "88c_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_09_05_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie14_20_CW{win}.root",
    "88c_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_09_38_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie15_20_CW{win}.root",
    "88c_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_10_12_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie16_20_CW{win}.root",
    "88c_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_10_46_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie17_20_CW{win}.root",
    "88c_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_11_19_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie18_20_CW{win}.root",
    "88c_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_11_53_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie19_20_CW{win}.root",
    "88c_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDA_HWPlus_D.2023_05_09_T.14_12_27_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie20_20_CW{win}.root",
}

iTEDB = {
    "888_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_49_01_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie1_25_CW{win}.root",
    "888_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_49_35_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie2_25_CW{win}.root",
    "888_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_50_08_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie3_25_CW{win}.root",
    "888_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_50_42_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie4_25_CW{win}.root",
    "888_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_51_16_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie5_25_CW{win}.root",
    "888_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_51_49_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie6_25_CW{win}.root",
    "888_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_52_23_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie7_25_CW{win}.root",
    "888_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_52_57_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie8_25_CW{win}.root",
    "888_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_53_31_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie9_25_CW{win}.root",
    "888_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_54_04_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie10_25_CW{win}.root",
    "888_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_54_38_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie11_25_CW{win}.root",
    "888_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_55_12_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie12_25_CW{win}.root",
    "888_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_55_45_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie13_25_CW{win}.root",
    "888_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_56_19_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie14_25_CW{win}.root",
    "888_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_56_53_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie15_25_CW{win}.root",
    "888_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_57_27_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie16_25_CW{win}.root",
    "888_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_58_00_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie17_25_CW{win}.root",
    "888_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_58_34_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie18_25_CW{win}.root",
    "888_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_59_08_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie19_25_CW{win}.root",
    "888_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.16_59_42_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie20_25_CW{win}.root",
    "888_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.17_00_15_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie21_25_CW{win}.root",
    "888_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.17_00_49_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie22_25_CW{win}.root",
    "888_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.17_01_23_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie23_25_CW{win}.root",
    "888_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.17_01_56_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie24_25_CW{win}.root",
    "888_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDB_HWPlus_D.2023_05_03_T.17_02_30_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie25_25_CW{win}.root",
    "88c_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDB_HWPlus_D.2023_05_08_T.15_58_09_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_5_CW{win}.root",
    "88c_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDB_HWPlus_D.2023_05_08_T.15_58_43_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_5_CW{win}.root",
    "88c_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDB_HWPlus_D.2023_05_08_T.15_59_16_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_5_CW{win}.root",
    "88c_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDB_HWPlus_D.2023_05_08_T.15_59_50_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_5_CW{win}.root",
    "88c_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDB_HWPlus_D.2023_05_08_T.16_00_24_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_5_CW{win}.root",
    "88c_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_13_38_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_20_CW{win}.root",
    "88c_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_14_12_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_20_CW{win}.root",
    "88c_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_14_45_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_20_CW{win}.root",
    "88c_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_15_19_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_20_CW{win}.root",
    "88c_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_15_53_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_20_CW{win}.root",
    "88c_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_16_26_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie6_20_CW{win}.root",
    "88c_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_17_00_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie7_20_CW{win}.root",
    "88c_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_17_34_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie8_20_CW{win}.root",
    "88c_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_18_08_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie9_20_CW{win}.root",
    "88c_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_18_41_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie10_20_CW{win}.root",
    "88c_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_19_15_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie11_20_CW{win}.root",
    "88c_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_19_48_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie12_20_CW{win}.root",
    "88c_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_20_22_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie13_20_CW{win}.root",
    "88c_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_20_56_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie14_20_CW{win}.root",
    "88c_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_21_29_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie15_20_CW{win}.root",
    "88c_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_22_03_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie16_20_CW{win}.root",
    "88c_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_22_37_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie17_20_CW{win}.root",
    "88c_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_23_10_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie18_20_CW{win}.root",
    "88c_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_23_44_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie19_20_CW{win}.root",
    "88c_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDB_HWPlus_D.2023_05_09_T.14_24_18_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie20_20_CW{win}.root",
}

iTEDC = {
    "888_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_05_39_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie1_25_CW{win}.root",
    "888_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_06_12_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie2_25_CW{win}.root",
    "888_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_06_46_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie3_25_CW{win}.root",
    "888_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_07_20_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie4_25_CW{win}.root",
    "888_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_07_53_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie5_25_CW{win}.root",
    "888_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_08_27_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie6_25_CW{win}.root",
    "888_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_09_01_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie7_25_CW{win}.root",
    "888_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_09_35_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie8_25_CW{win}.root",
    "888_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_10_08_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie9_25_CW{win}.root",
    "888_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_10_42_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie10_25_CW{win}.root",
    "888_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_11_16_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie11_25_CW{win}.root",
    "888_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_11_49_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie12_25_CW{win}.root",
    "888_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_12_23_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie13_25_CW{win}.root",
    "888_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_12_57_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie14_25_CW{win}.root",
    "888_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_13_31_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie15_25_CW{win}.root",
    "888_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_14_04_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie16_25_CW{win}.root",
    "888_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_14_38_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie17_25_CW{win}.root",
    "888_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_15_12_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie18_25_CW{win}.root",
    "888_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_15_46_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie19_25_CW{win}.root",
    "888_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_16_19_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie20_25_CW{win}.root",
    "888_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_16_53_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie21_25_CW{win}.root",
    "888_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_17_27_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie22_25_CW{win}.root",
    "888_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_18_00_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie23_25_CW{win}.root",
    "888_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_18_34_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie24_25_CW{win}.root",
    "888_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDC_HWPlus_D.2023_05_03_T.17_19_08_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie25_25_CW{win}.root",
    "88c_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDC_HWPlus_D.2023_05_08_T.16_01_23_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_5_CW{win}.root",
    "88c_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDC_HWPlus_D.2023_05_08_T.16_01_56_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_5_CW{win}.root",
    "88c_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDC_HWPlus_D.2023_05_08_T.16_02_30_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_5_CW{win}.root",
    "88c_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDC_HWPlus_D.2023_05_08_T.16_03_04_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_5_CW{win}.root",
    "88c_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDC_HWPlus_D.2023_05_08_T.16_03_38_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_5_CW{win}.root",
    "88c_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_27_32_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_20_CW{win}.root",
    "88c_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_28_06_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_20_CW{win}.root",
    "88c_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_28_39_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_20_CW{win}.root",
    "88c_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_29_13_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_20_CW{win}.root",
    "88c_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_29_47_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_20_CW{win}.root",
    "88c_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_30_20_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie6_20_CW{win}.root",
    "88c_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_30_54_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie7_20_CW{win}.root",
    "88c_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_31_28_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie8_20_CW{win}.root",
    "88c_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_32_02_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie9_20_CW{win}.root",
    "88c_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_32_35_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie10_20_CW{win}.root",
    "88c_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_33_09_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie11_20_CW{win}.root",
    "88c_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_33_43_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie12_20_CW{win}.root",
    "88c_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_34_16_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie13_20_CW{win}.root",
    "88c_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_34_50_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie14_20_CW{win}.root",
    "88c_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_35_24_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie15_20_CW{win}.root",
    "88c_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_35_57_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie16_20_CW{win}.root",
    "88c_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_36_31_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie17_20_CW{win}.root",
    "88c_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_37_05_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie18_20_CW{win}.root",
    "88c_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_37_39_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie19_20_CW{win}.root",
    "88c_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDC_HWPlus_D.2023_05_09_T.14_38_12_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie20_20_CW{win}.root",
}

iTEDD = {
    "888_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_22_58_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie1_25_CW{win}.root",
    "888_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_23_31_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie2_25_CW{win}.root",
    "888_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_24_05_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie3_25_CW{win}.root",
    "888_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_24_39_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie4_25_CW{win}.root",
    "888_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_25_12_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie5_25_CW{win}.root",
    "888_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_25_46_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie6_25_CW{win}.root",
    "888_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_26_20_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie7_25_CW{win}.root",
    "888_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_26_53_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie8_25_CW{win}.root",
    "888_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_27_27_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie9_25_CW{win}.root",
    "888_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_28_01_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie10_25_CW{win}.root",
    "888_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_28_35_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie11_25_CW{win}.root",
    "888_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_29_08_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie12_25_CW{win}.root",
    "888_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_29_42_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie13_25_CW{win}.root",
    "888_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_30_16_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie14_25_CW{win}.root",
    "888_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_30_50_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie15_25_CW{win}.root",
    "888_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_31_23_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie16_25_CW{win}.root",
    "888_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_31_57_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie17_25_CW{win}.root",
    "888_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_32_31_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie18_25_CW{win}.root",
    "888_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_33_04_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie19_25_CW{win}.root",
    "888_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_33_38_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie20_25_CW{win}.root",
    "888_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_34_12_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie21_25_CW{win}.root",
    "888_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_34_46_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie22_25_CW{win}.root",
    "888_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_35_19_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie23_25_CW{win}.root",
    "888_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_35_53_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie24_25_CW{win}.root",
    "888_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_03/Cs137_iTEDD_HWPlus_D.2023_05_03_T.17_36_27_C.itedABCD_lab_2023.02.22_4.0v_888_30s_serie25_25_CW{win}.root",
    "88c_1":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDD_HWPlus_D.2023_05_08_T.16_05_24_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_5_CW{win}.root",
    "88c_2":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDD_HWPlus_D.2023_05_08_T.16_05_57_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_5_CW{win}.root",
    "88c_3":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDD_HWPlus_D.2023_05_08_T.16_06_31_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_5_CW{win}.root",
    "88c_4":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDD_HWPlus_D.2023_05_08_T.16_07_05_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_5_CW{win}.root",
    "88c_5":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_08/Cs137_iTEDD_HWPlus_D.2023_05_08_T.16_07_38_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_5_CW{win}.root",
    "88c_6":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_20_38_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie1_20_CW{win}.root",
    "88c_7":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_21_12_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie2_20_CW{win}.root",
    "88c_8":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_21_45_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie3_20_CW{win}.root",
    "88c_9":  lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_22_19_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie4_20_CW{win}.root",
    "88c_10": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_22_53_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie5_20_CW{win}.root",
    "88c_11": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_23_27_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie6_20_CW{win}.root",
    "88c_12": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_24_00_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie7_20_CW{win}.root",
    "88c_13": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_24_34_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie8_20_CW{win}.root",
    "88c_14": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_25_08_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie9_20_CW{win}.root",
    "88c_15": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_25_42_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie10_20_CW{win}.root",
    "88c_16": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_26_15_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie11_20_CW{win}.root",
    "88c_17": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_26_49_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie12_20_CW{win}.root",
    "88c_18": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_27_23_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie13_20_CW{win}.root",
    "88c_19": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_27_57_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie14_20_CW{win}.root",
    "88c_20": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_28_30_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie15_20_CW{win}.root",
    "88c_21": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_29_04_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie16_20_CW{win}.root",
    "88c_22": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_29_38_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie17_20_CW{win}.root",
    "88c_23": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_30_11_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie18_20_CW{win}.root",
    "88c_24": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_30_45_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie19_20_CW{win}.root",
    "88c_25": lambda win: f"/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/data_2023_05_09/Cs137_iTEDD_HWPlus_D.2023_05_09_T.15_31_19_C.itedABCD_lab_custom_2023.02.22_4.0v_887_30s_serie20_20_CW{win}.root",
}

In [7]:
def get_resolution(cell):
    
    TH1D = cell.TH1D()
    
    TH1D.GetXaxis().SetRange(TH1D.FindBin(100),TH1D.FindBin(400))
    
    MaxBin   = TH1D.FindBin(TH1D.GetMaximumBin())
    
    ADC_Low  = MaxBin-50
    ADC_High = MaxBin+40

    gaussFit = ROOT.TF1("gaussFit", "pol2(0)+gaus(3)", ADC_Low, ADC_High)
    gaussFit.SetParameters(1000,-1,0,TH1D.GetMaximum(),MaxBin,10)
    TH1D.Fit(gaussFit,"QR")
    
    sigma = abs(gaussFit.GetParameter(5))
    centroid_ch = gaussFit.GetParameter(4)
    
    x1 = cell(centroid_ch+sigma*numpy.sqrt(2*numpy.log(2)))
    x2 = cell(centroid_ch-sigma*numpy.sqrt(2*numpy.log(2)))
    centroid = cell(centroid_ch)
        
    fwhm = x1-x2
            
    return sigma*numpy.sqrt(2*numpy.log(2))*2/centroid_ch*100, (fwhm/centroid)*100, centroid

In [8]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)
    
    l4 = uproot.open(f"../../data/nTOF_March2022/888/CW100ns/Resolutions_Cs137_CenterScatter_iTED{cell.iTED()}_8.8.8_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    if l4*0.9 < l2 < l4*1.1:
        latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
    else:
        latex.DrawText(0.7, 0.75, "->R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
        
    latex.DrawText(0.7, 0.65, "R_E(old): {:.2f}%".format(l4))
    latex.DrawText(0.7, 0.6, "Change: {:.2f}%".format(l2-l4))
            
    return canvas

In [10]:
entries = []

for iTED in tqdm.tqdm([0,1,2,3]):
    for Crystal in tqdm.tqdm([0,1,2,3,4]):
        for Configuration in ["888", "88c"]:
            for CW in [100,150,200,250]:
                for i_run in range(1,25+1):
                    
                    cryst_code = ["A", "B", "C", "D"][iTED]+str(Crystal)
                                        
                    [iTEDA,iTEDB,iTEDC,iTEDD][iTED][f"{Configuration}_{i_run}_{CW}"] = [iTEDA,iTEDB,iTEDC,iTEDD][iTED][f"{Configuration}_{i_run}"](CW)
                                                    
                    spectr = spectrum(
                        [iTEDA,iTEDB,iTEDC,iTEDD][iTED][f"{Configuration}_{i_run}"](CW),
                        ["A","B","C","D"][iTED], 
                        Crystal, 
                        Configuration, 
                        CW, 
                        calibrations_df.query("crystal == @cryst_code & cw == '100'").iloc[0][[0,1,2]],
                        i_run
                    )
                                    
                    entries.append(
                        pandas.DataFrame({
                            "resolution": get_resolution(spectr)[1],
                            "fit": abs(get_resolution(spectr)[2]-662)/662*100,
                            "cps": spectr.Rate(30),
                            "alpha": spectr.Alpha(30),
                            "iTED": ["A", "B", "C", "D"][iTED],
                            "crystal": Crystal,
                            "cw": CW,
                            "configuration": Configuration,
                            "run": i_run,
                            "obj": spectr,
                        }, index=[0])
                    )
                    
entries_df = pandas.concat(entries, ignore_index=True)

100%|██████████████████| 4/4 [00:53<00:00, 13.44s/it]


## Analysis

### Resolution

#### per iTED per crystal per configuration per window

In [11]:
entries_df.groupby(["iTED","cw","configuration"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

In [12]:
print("The distribution of the resolution standard deviation in the runs:")
entries_df.groupby(["iTED","crystal","cw","configuration"]).resolution.describe().drop(['count', '25%', '50%', '75%'], axis=1)["std"].describe().drop(['count', 'mean', 'std'], axis=0)

The distribution of the resolution standard deviation in the runs:


min    0.046405
25%    0.068736
50%    0.088982
75%    0.117384
max    0.642811
Name: std, dtype: float64

#### per configuration

In [13]:
entries_df.groupby(["configuration"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

configuration,888,88c
mean,6.912598,6.796691
min,5.443369,5.367163
25%,6.524867,6.370670
50%,6.955731,6.782947
75%,7.246383,7.171045
max,9.247311,9.739220


#### per window

In [17]:
entries_df.groupby(["cw"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

cw,100,150,200,250
mean,6.949815,6.880050,6.819506,6.769207
min,5.488765,5.430275,5.382213,5.367163
25%,6.542288,6.495574,6.436399,6.351605
50%,6.916190,6.836750,6.797165,6.775565
75%,7.273542,7.224771,7.195815,7.165071
max,9.739220,9.736662,9.718043,9.717470


#### per iTED

In [15]:
entries_df.groupby(["iTED"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

iTED,A,B,C,D
mean,6.414433,7.031052,7.168048,6.805044
min,5.367163,6.540301,5.592214,6.135096
25%,5.767904,6.853733,6.423626,6.525794
50%,6.344660,7.019963,6.988982,6.695925
75%,6.656698,7.202743,7.988073,7.080802
max,8.426365,7.567662,9.362220,9.739220


#### per parameters (conf and window)

In [16]:
entries_df.groupby(["configuration","cw"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

### Fit

#### per parameters

In [24]:
entries_df.groupby(["configuration","cw"]).fit.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

#### per run

In [25]:
entries_df.groupby(["iTED","crystal","configuration"]).fit.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

### Count rate using integral

In [ ]:
print(entries_df.groupby(["configuration","cw"]).resolution.describe().drop(['count', 'std'], axis=1).T.to_latex(float_format="{:.2f}".format))

In [33]:
entries_df.query("configuration == '88c'").groupby(["iTED"]).cps.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

iTED,A,B,C,D
mean,4740.134000,3920.882733,4606.956133,4236.435200
min,2598.833333,1777.433333,2695.066667,2996.566667
25%,3932.958333,1952.091667,3382.483333,3597.858333
50%,3977.966667,3403.083333,3800.766667,3979.133333
75%,5545.683333,5264.616667,5340.991667,4628.666667
max,9581.000000,7431.333333,7935.300000,6090.866667


In [26]:
entries_df.groupby(["iTED","crystal"]).cps.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

### Alpha activity rate using 1600keV < E < 2800keV

In [52]:
entries_df.query("configuration == '88c' & cw == 250").groupby(["iTED","crystal"]).alpha.describe().drop(['count', 'std'], axis=1).T

iTED             A                                                          B  \
crystal          0           1           2          3           4           0   
mean     24.181333  157.682667  117.384000  36.720000  222.200000  143.976000   
min      18.066667  120.700000   86.600000  27.433333  169.933333  133.966667   
25%      23.233333  156.133333  117.533333  35.966667  222.566667  141.533333   
50%      24.266667  159.866667  118.700000  37.166667  225.766667  144.333333   
75%      25.233333  162.533333  120.500000  38.233333  227.633333  147.533333   
max      28.100000  169.900000  123.733333  39.766667  231.900000  153.166667   

iTED                                                              C  \
crystal           1           2           3           4           0   
mean     124.041333  204.449333  221.008000  215.386667  153.816000   
min      116.500000  187.600000  201.966667  199.466667  146.033333   
25%      121.733333  201.866667  218.233333  210.800000  151.500000   
50%      125.666667  204.900000  221.966667  218.400000  153.700000   
75%      127.033333  207.566667  226.233333  220.066667  156.233333   
max      130.033333  214.900000  230.766667  224.866667  159.133333   

iTED                                                              D  \
crystal           1           2           3           4           0   
mean     225.545333  230.120000  196.445333  192.528000  151.782667   
min      214.633333  226.366667  185.933333  172.466667  143.033333   
25%      220.200000  227.833333  194.566667  186.166667  150.300000   
50%      227.133333  229.166667  197.033333  196.100000  151.933333   
75%      230.366667  231.700000  200.133333  200.966667  154.933333   
max      235.300000  237.633333  204.500000  206.933333  157.633333   

iTED                                                     
crystal           1           2           3           4  
mean     219.501333  240.789333  227.262667  231.866667  
min      200.533333  228.266667  214.600000  217.866667  
25%      216.000000  237.966667  223.033333  226.900000  
50%      220.433333  241.966667  227.900000  231.800000  
75%      223.533333  244.200000  231.733333  236.633333  
max      228.400000  250.300000  237.900000  241.033333

In [32]:
entries_df.groupby(["iTED","crystal"]).alpha.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

## Draw fits

In [ ]:
#for index, row in entries_df.iterrows():
#    TH1D_draw(row.obj).Draw()